In [1]:
from keras.utils import np_utils
import numpy as np
from matplotlib import pyplot
import seaborn as sns
%matplotlib inline

# Display progress logs on stdout
#import logging
#logging.basicConfig(level=logging.DEBUG,
#                    format='%(asctime)s %(levelname)s %(message)s')

from sklearn.datasets import load_files
import pandas as pd
pd.set_option("display.max_colwidth", 75)

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)

Using TensorFlow backend.
/home/tutysara/anaconda2/envs/dog-project/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
basedir="/home/tutysara/src/myprojects/dog-project/dogImages"
train_idx_path = basedir+ "/train_list.txt"
valid_idx_path = basedir+ "/valid_list.txt"
test_idx_path = basedir+ "/test_list.txt"


col_names = ["X", "y"]
train_data_df = pd.read_csv(train_idx_path, sep=" ", header=None, names=col_names)
train_data_df.y = train_data_df.y-1
valid_data_df = pd.read_csv(valid_idx_path, sep=" ", header=None, names=col_names)
valid_data_df.y = valid_data_df.y-1
test_data_df = pd.read_csv(test_idx_path, sep=" ", header=None, names=col_names)
test_data_df.y = test_data_df.y-1

In [3]:
test_data_df.head()

,X,y
0,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05148.jpg,74
1,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05143.jpg,74
2,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05164.jpg,74
3,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05129.jpg,74
4,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05149.jpg,74


In [4]:
print(train_data_df.shape)
print(valid_data_df.shape)
print(test_data_df.shape)

print(train_data_df.shape[0] + valid_data_df.shape[0] + test_data_df.shape[0])
sample_df = test_data_df[:5]
sample_df

(6680, 2)
(835, 2)
(836, 2)
8351


,X,y
0,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05148.jpg,74
1,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05143.jpg,74
2,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05164.jpg,74
3,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05129.jpg,74
4,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05149.jpg,74


In [5]:
# convert and load images
from keras.preprocessing import image                  
from tqdm import tqdm
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in img_paths]
    return np.vstack(list_of_tensors)

In [6]:
#definition of data generator
def df_data_generator(df, batch_size=32, num_classes=3, shuffle=False, basedir=".", transformer=None):    
    while True:
        if shuffle:
            df = df.sample(frac=1)
            
        X_file_name = df.X.apply(lambda x: basedir+"/"+x) 
        y = np_utils.to_categorical(df.y, num_classes)
    # infinitely serve batches
        max_range = (1+ df.shape[0]//batch_size)
        for i in range(max_range):
            if transformer:
                yield transformer(X_file_name[i*batch_size : (i+1)*batch_size]).astype('float32'), y[i*batch_size : (i+1)*batch_size]
            else:
                yield X_file_name[i*batch_size : (i+1)*batch_size], y[i*batch_size : (i+1)*batch_size]
                

from keras.applications.mobilenet import MobileNet
import keras.applications.mobilenet as mobilenet
def transformer(x):
    return mobilenet.preprocess_input(paths_to_tensor(x))


In [7]:
bnf_train_name = 'bottleneck_features_mobilenet_train'
bnf_valid_name = 'bottleneck_features_mobilenet_valid'
bnf_test_name = 'bottleneck_features_mobilenet_test' 

In [8]:
import bcolz
def bcolz_prediction_writer(gen, steps, model, dirname ):
    X = None; y = None
    data_dir = dirname + "_data.bclz"
    label_dir = dirname + "_labels.bclz"
    for i in range(steps):
        X_batch, y_batch = next(gen)
        X_out = model.predict(X_batch)
        if i== 0:
            X = bcolz.carray(X_out, rootdir=data_dir, mode='w')
            y = bcolz.carray(y_batch, rootdir=label_dir, mode='w') 
        else:
            X.append(X_out)
            y.append(y_batch)
    X.flush()
    y.flush()
    return X, y

In [9]:
from keras.applications.mobilenet import MobileNet
import time

# load train, test, and validation datasets
batch_size=32
train_gen = df_data_generator(train_data_df, batch_size=batch_size, transformer=transformer, basedir=basedir, num_classes=133)
valid_gen = df_data_generator(valid_data_df, batch_size=batch_size, transformer=transformer, basedir=basedir, num_classes=133)
test_gen = df_data_generator(test_data_df, batch_size=batch_size, transformer=transformer, basedir=basedir, num_classes=133)



mobilenet_feature_ext = MobileNet(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
mobilenet_feature_ext._make_predict_function()

s= time.time()
!rm -rf 'bottleneck_features_mobilenet_valid*bclz'
bnf_valid_data,  bnf_valid_labels = bcolz_prediction_writer(gen=valid_gen, 
                        steps=(1+(valid_data_df.shape[0]//batch_size)),
                        model=mobilenet_feature_ext,
                        dirname=bnf_valid_name)
print("Took {} seconds to calculate bnf_valid_data".format(time.time()-s))

s= time.time()
!rm -rf 'bottleneck_features_mobilenet_test*bclz'
bnf_test_data,  bnf_test_labels = bcolz_prediction_writer(gen=test_gen, 
                        steps=(1+(test_data_df.shape[0]//batch_size)),
                        model=mobilenet_feature_ext,
                        dirname=bnf_test_name)
print("Took {} seconds to calculate bnf_test_data".format(time.time()-s))

s= time.time()
!rm -rf 'bottleneck_features_mobilenet_train*bclz'
bnf_train_data,  bnf_train_labels = bcolz_prediction_writer(gen=train_gen, 
                        steps=(1+(train_data_df.shape[0]//batch_size)),
                        model=mobilenet_feature_ext,
                        dirname=bnf_train_name)
print("Took {} seconds to calculate bottleneck_features_mobilenet_train".format(time.time()-s))


Took 12.44444465637207 seconds to calculate bnf_valid_data
Took 11.485620260238647 seconds to calculate bnf_test_data
Took 96.61987161636353 seconds to calculate bottleneck_features_mobilenet_train


In [10]:
print(bnf_valid_data.shape)
print(bnf_test_data.shape)
print(bnf_train_data.shape)


(835, 7, 7, 1024)
(836, 7, 7, 1024)
(6680, 7, 7, 1024)


In [11]:
print(bnf_valid_labels.shape)
print(bnf_test_labels.shape)
print(bnf_train_labels.shape)

(835, 133)
(836, 133)
(6680, 133)


In [14]:
bnf_valid_data = bcolz.carray(rootdir='bottleneck_features_mobilenet_valid_data.bclz', mode='r')
bnf_test_data = bcolz.carray(rootdir='bottleneck_features_mobilenet_test_data.bclz', mode='r')
bnf_train_data = bcolz.carray(rootdir='bottleneck_features_mobilenet_train_data.bclz', mode='r')

print(bnf_valid_data.shape)
print(bnf_test_data.shape)
print(bnf_train_data.shape)

(835, 7, 7, 1024)
(836, 7, 7, 1024)
(6680, 7, 7, 1024)


In [15]:
bnf_valid_labels = bcolz.carray(rootdir='bottleneck_features_mobilenet_valid_labels.bclz', mode='r')
bnf_test_labels = bcolz.carray(rootdir='bottleneck_features_mobilenet_test_labels.bclz', mode='r')
bnf_train_labels = bcolz.carray(rootdir='bottleneck_features_mobilenet_train_labels.bclz', mode='r')

print(bnf_valid_labels.shape)
print(bnf_test_labels.shape)
print(bnf_train_labels.shape)

(835, 133)
(836, 133)
(6680, 133)


In [16]:
def bnf_data_generator(bnf_data, bnf_labels, batch_size=32):
    while True:
        max_range = (1 + bnf_data.shape[0]//batch_size)
        for i in range(max_range):
            yield bnf_data[i*batch_size : (i+1)*batch_size], bnf_labels[i*batch_size : (i+1)*batch_size]                

In [17]:
## make a generator of loaded data
batch_size = 256
train_gen_fex =bnf_data_generator(bnf_train_data, bnf_train_labels, batch_size=batch_size)
valid_gen_fex =bnf_data_generator(bnf_valid_data, bnf_valid_labels, batch_size=batch_size)
test_gen_fex = bnf_data_generator(bnf_test_data, bnf_test_labels, batch_size=batch_size) 

In [18]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import regularizers

top_model = Sequential()
#model.add(Flatten(input_shape=train_data.shape[1:]))
top_model.add(GlobalAveragePooling2D(input_shape=(7, 7, 1024)))
top_model.add(Dropout(0.2))
top_model.add(Dense(512, activation='relu'))
top_model.add(Dropout(0.2))
top_model.add(Dense(133, activation='softmax'))

top_model.compile(optimizer='adam',
              loss='categorical_crossentropy', metrics=['accuracy'])

top_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 133)               68229     
Total params: 593,029
Trainable params: 593,029
Non-trainable params: 0
_________________________________________________________________


In [19]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.topmodel.mobilenet.hdf5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
top_model.fit_generator(train_gen_fex,
          steps_per_epoch= (1 + (train_data_df.shape[0]// batch_size)),
          epochs=25,
          validation_data=valid_gen_fex,
          validation_steps= (1 + (valid_data_df.shape[0] // batch_size)),
          callbacks=[early_stopping, checkpointer])

Epoch 1/25
27/27 [==============================] - 3s 124ms/step - loss: 5.8226 - acc: 1.4486e-04 - val_loss: 4.8263 - val_acc: 0.0551
Epoch 2/25
27/27 [==============================] - 3s 97ms/step - loss: 4.8337 - acc: 0.0336 - val_loss: 4.6734 - val_acc: 0.0862
Epoch 3/25
27/27 [==============================] - 3s 101ms/step - loss: 4.6861 - acc: 0.0594 - val_loss: 4.2980 - val_acc: 0.1150
Epoch 4/25
27/27 [==============================] - 3s 98ms/step - loss: 4.2977 - acc: 0.0963 - val_loss: 3.6955 - val_acc: 0.2335
Epoch 5/25
27/27 [==============================] - 3s 103ms/step - loss: 3.7658 - acc: 0.1502 - val_loss: 3.0514 - val_acc: 0.3078
Epoch 6/25
27/27 [==============================] - 3s 101ms/step - loss: 3.1970 - acc: 0.2055 - val_loss: 2.5161 - val_acc: 0.4168
Epoch 7/25
27/27 [==============================] - 3s 100ms/step - loss: 2.7505 - acc: 0.2756 - val_loss: 2.0976 - val_acc: 0.5114
Epoch 8/25
27/27 [==============================] - 3s 99ms/step - loss: 2

In [21]:
def prediction_from_gen(gen, steps, model, dirname ):
    y_true = None; y_pred = None
    yt_dir = dirname + "_y_true.bclz"
    yp_dir = dirname + "_y_pred.bclz"
    for i in range(steps):
        X_batch, y_batch = next(gen)
        y_out = model.predict(X_batch)
        if i== 0:
            y_true = bcolz.carray(y_batch, rootdir=yt_dir, mode='w')
            y_pred = bcolz.carray(y_out, rootdir=yp_dir, mode='w') 
        else:
            y_true.append(y_batch)
            y_pred.append(y_out)
    y_true.flush()
    y_pred.flush()
    return y_true, y_pred

y_true, y_pred = prediction_from_gen(gen=test_gen_fex,
                                     steps=(1 + (test_data_df.shape[0] // batch_size)),
                                     model=top_model,
                                     dirname="bottleneck_features_mobilenet_test"
                                    )


In [24]:
print(y_true.shape)
print(y_pred.shape)

(836, 133)
(836, 133)


In [25]:
# report test accuracy
test_accuracy = 100*np.sum(np.argmax(y_pred, axis=1)==np.argmax(y_true, axis=1))/len(y_true)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 80.6220%


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1)
top_model.fit(train_data[:250], train_labels[:250],
          epochs=25,
          batch_size=64,
          validation_data=(valid_data[:150], valid_labels[:150]),
          callbacks=[early_stopping])

In [26]:
# fine tune on full model
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint

mobilenet_model = MobileNet(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# CREATE AN "REAL" MODEL FROM Mobilenet
# BY COPYING ALL THE LAYERS OF Mobilenet
model = Sequential()
for l in mobilenet_model.layers:
    model.add(l)


# CONCATENATE THE TWO MODELS
model.add(top_model)

# LOCK THE TOP CONV LAYERS
for layer in model.layers:
    layer.trainable = False

# COMPILE THE MODEL
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 32)      0         
__________

In [ ]:
# load train, test, and validation datasets
batch_size=256
train_gen = df_data_generator(train_data_df, batch_size=batch_size, transformer=transformer, basedir=basedir, num_classes=133)
valid_gen = df_data_generator(valid_data_df, batch_size=batch_size, transformer=transformer, basedir=basedir, num_classes=133)
test_gen = df_data_generator(test_data_df, batch_size=batch_size, transformer=transformer, basedir=basedir, num_classes=133)


from keras.callbacks import EarlyStopping, ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.fullmodel.mobilenet.hdf5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1)
model.fit_generator(train_gen,
          steps_per_epoch= (1+ train_data_df.shape[0]// batch_size),
          epochs=2,
          validation_data=valid_gen,
          validation_steps= (1+ valid_data_df.shape[0] // batch_size),
          callbacks=[early_stopping, checkpointer])

Epoch 1/2
27/27 [==============================] - 87s 3s/step - loss: 2.1309 - acc: 0.4690 - val_loss: 0.6342 - val_acc: 0.8120
Epoch 2/2
25/27 [==========================>...] - ETA: 4s - loss: 1.8657 - acc: 0.4852

In [ ]:

y_true, y_pred = prediction_from_gen(gen=test_gen,
                                     steps=(1 + (test_data_df.shape[0] // batch_size)),
                                     model=model,
                                     dirname="mobilenet_test"
                                    )